In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
import gc

In [ ]:
# Load the data
spectrum_2D_dir = '../../../primary_data/main_mosaic_6by5/'
spectrum_2D_list = sorted(os.listdir(spectrum_2D_dir))
# Select tiles
indices = [0,1,8]
# Select colormap
cmap = plt.get_cmap('nipy_spectral')
size = [1024, 1024, 250]


In [ ]:
def MinMax(arr):
    arr = (arr - np.min(arr)) / (np.max(arr) - np.min(arr))
    return arr

cmap_idx = np.linspace(0.1, 0.9, size[2])

sel_files = []
for i in range(len(indices)):
    sel_files.append(spectrum_2D_list[indices[i]])

In [ ]:
for idx, file in enumerate(sel_files):
    d = np.load(spectrum_2D_dir + file)['spectrum_2D'].reshape((size[0],size[1],size[2]))
    for z in range(d.shape[2]):
        d[:,:,z] = MinMax(d[:,:,z])
    d = d.astype('float32')
    d = d.reshape((size[0]*size[1], size[2]))
    
    r = np.zeros((size[0] * size[1], size[2]))
    g = np.zeros((size[0] * size[1], size[2]))
    b = np.zeros((size[0] * size[1], size[2]))
    a = np.zeros((size[0] * size[1], size[2]))

    for z in range(d.shape[1]):
        r[:,z] = cmap(cmap_idx[z])[0]
        g[:,z] = cmap(cmap_idx[z])[1]
        b[:,z] = cmap(cmap_idx[z])[2]
        a[:,z] = d[:,z]
        
    rgb_arr = np.zeros((size[0] * size[1], size[2], 4)).astype('float32')
    rgb_arr[:,:,0] = r
    rgb_arr[:,:,1] = g
    rgb_arr[:,:,2] = b
    rgb_arr[:,:,3] = a
    
    rgb_arr = rgb_arr.reshape((size[0], size[1], size[2], 4))
    d = d.reshape((size[0], size[1], size[2]))
    
    d = d.transpose((1,0,2))
    d = np.flip(d, axis = 1)
    rgb_arr = rgb_arr.transpose((1,0,2,3))
    rgb_arr = np.flip(rgb_arr, axis = 1)
    
    f = plt.figure(figsize = (10,10), dpi = 300, layout =  'tight')
    ax = plt.axes(projection = '3d')
    ax.voxels(d, facecolors = rgb_arr)
    ax.view_init(15,15, vertical_axis ='y')
    ax.invert_zaxis()
    plt.axis('off')
    plt.savefig('../png/3D_Plot_Tile_' + '%02d' %(indices[idx]) + '.png')
    plt.show()
    gc.collect()